# Recolectar datos a través de Web Scraping ⛏️

<strong>Objetivos:</strong>
+ Extraer de Wikipedia una tabla HTML con registros de lanzamientos de cohetes Falcon 9.
+ Parsear la tabla y convertirla en un DataFrame de Pandas.

---

## Importar librerías y definir funciones auxiliares

In [5]:
# Importar.
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

# Para ignorar Warnings.
import warnings
warnings.filterwarnings('ignore')

Definir funciones auxiliares para procesar la tabla HTML extraída de la web.

In [6]:
def date_time(table_cells):
    '''
    Esta función devuelve un string con la fecha y hora extraídas del parámetro.
    La función utiliza una list comprehension para crear una nueva lista llamada 'data_time' con los contenidos de cada celda, aplicando el método strip() para eliminar cualquier espacio en blanco al principio o al final del string.
    Luego, utiliza la función 'list(table_cells.strings)' para extraer solo los strings dentro de las celdas (ignorando los elementos html) y devuelve solo los dos primeros elementos de la lista 'data_time' con [0:2].
    '''
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    '''
    Esta función devuelve un string con la versión del booster (propulsor) extraída del parámetro.
    La función utiliza una list comprehension y la función enumerate() para crear una nueva lista 'booster_version' con los contenidos de cada celda, solo si su índice es par (i%2==0).
    Luego, utiliza la función 'table_cells.strings' para extraer solo los strings dentro de las celdas (ignorando los elementos html)
    Finalmente, con la función 'join()' une todos los elementos de la lista 'booster_version' en un solo string, eliminando el ultimo elemento con '[0:-1]'.
    '''
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    '''
    Esta función devuelve un string con el estado del aterrizaje extraído del parámetro.
    La función utiliza una list comprehension para recorrer los strings dentro de 'table_cells' y devuelve el primer string dentro de esa celda.
    '''
    out = [i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    '''
    Esta función devuelve un string con el peso en kg si existe o 0 si no existe.
    La función utiliza 'unicodedata' para normalizar el texto dentro del parámetro, eliminando cualquier caracter no compatible con el conjunto de caracteres Unicode 'NFKD', y almacenando el resultado en una variable 'mass'. Luego, utiliza el método 'strip' para eliminar cualquier espacio en blanco al inicio o al final del string.
    Luego la función verifica si existe un valor dentro de 'mass'. Si es así, utiliza el método 'find' para buscar el string 'kg' y obtener su posición. Después se extrae un substring desde el inicio hasta la posición encontrada mas 2, es decir, dos caracteres después de 'kg', y se guarda en una nueva variable 'new_mass'. Si 'mass' es False (no existe un valor dentro), 'new_mass' es 0.
    '''
    mass = unicodedata.normalize('NFKD', table_cells.text).strip()
    if mass:
        mass.find('kg')
        new_mass = mass[0:mass.find('kg')+2]
    else:
        new_mass = 0
    return new_mass

def extract_column_from_header(row):
    '''
    Esta función toma como entrada la fila de una tabla (en formato de un elemento 'row' de BeautifulSoup) y devuelve el nombre de la columna asociada a esa fila, eliminando ciertos elementos en el proceso.
    En primer lugar, si existe un elemento 'br' en la fila, se extrae. Luego si existe un elemento 'a' también se extrae. Y finalmente si existe un elemento 'sup', también se extrae.
    Después se concatenan todos los elementos restantes con espacios en blanco para formar el nombre de la columna, se elimina cualquier espacio en blanco de los bordes y se verifica si el resultado es un número. Si el resultado no es un número se retorna ese string, si es un número se sale de la función sin retornar un valor.
    '''
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name  

In [7]:
# Url a raspar:
static_url = 'https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922'

> **NOTA:**
La Wiki a raspar debe pertenecer a la fecha del 9 de junio de 2021. Así lo establecen los creadores de este proyecto para mantener consistencia entre las tareas.

A continuación, solicitar la página HTML de la URL anterior para obtener un objeto <code>response</code>.

## Tareas

### Tarea 1: Solicitar la Wiki sobre lanzamientos de cohetes Falcon 9

Primero, utilizar el método HTTP GET para solicitar la página HTML como una respuesta HTTP.

In [8]:
# Usar el método requests.get() con la static_url definida anteriormente.
# Asignar la respuesta a un objeto response.
response = requests.get(static_url)

Crear un objeto <code>BeautifulSoup</code> a partir de <code>response</code>.

In [9]:
# Usar BeautifulSoup() para crear objeto a partir del texto de la respuesta.
soup = BeautifulSoup(response.text, 'html.parser')

Por último, imprimir el título de la página para verificar si el objeto BeautifulSoup se creó correctamente.

In [10]:
# Usar atributo soup.title.
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

Todo 👌🏻

### Tarea 2: Extraer todos los nombres de las columnas/variables del header de la tabla HTML

Para obtener los nombres de las columnas del header de la tabla HTML correcta, primero hay que extraer todas las tablas de la página wiki.

In [11]:
# En el objeto de BeautifulSoup, usar la función find_all con el elemento 'table'.
# Asignar el resultado a la lista 'html_tables'.
html_tables = soup.find_all('table')

Resulta que a partir de la tercera tabla comienzan los registros de lanzamiento de Falcon 9. Validar.

In [12]:
# Imprimir la tercera tabla y verificar contenido.
# Asignar la tecera tabla a una nueva variable que la referencie como la primera tabla de lanzamientos.
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

Ahora es momento de iterar a través de los elementos <code>`<th>`</code> de la primera tabla de lanzamientos y aplicarles la función auxiliar <code>extract_column_from_header()</code> para extraer uno por uno los nombres de las columnas.

In [13]:
# En first_launch_table aplicar find_all() con el elemento th y asignar resultado a una variable 'th_elements'.
# Iterar cada elemento th obtenido y aplicar la función extract_column_from_header() para obtener el nombre de la columna que asocia.
# Si el nombre de la columna no es None y len(nombre) > 0, agregarlo a una lista llamada 'column_names'.

column_names = []

th_elements = first_launch_table.find_all('th')

for th in th_elements:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

Verificar los nombres extraídos.

In [14]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


### Tarea 3: Crear DataFrame parseando los registros de las tablas de lanzamientos

Para esta tarea se debe crear un diccionario vacío con claves que sean los nombres de las columnas ya extraídas. Más tarde, este diccionario se convertirá en un DataFrame de Pandas.

In [15]:
launch_dict = dict.fromkeys(column_names)

# Remover columnas irrelevantes.
del launch_dict['Date and time ( )']

# Inicializar el launch_dict para que cada valor sea una lista vacía.
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Añador nuevas columnas de interés, también con listas vacías como valores.
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

Ahora, solo se necesita completar el <code>launch_dict</code> con los registros de lanzamiento extraídos de las filas de las tablas apropiadas.

Por lo general, es probable que las tablas HTML en las páginas Wiki contengan anotaciones inesperadas y otros tipos de ruidos, como enlaces de referencia <code>B0004.1[8]</code>, valores faltantes <code>N/A [e]</code>, formato inconsistente, etc.

Para simplificar el proceso, a continuación hay un fragmento de código incompleto que ayudará a completar el launch_dict. Responder a los TODOs para parsear las tablas.

In [16]:
extracted_row = 0
'''
A continuación, el primer ciclo for recorre todas las tablas en el documento HTML que tienen la clase 'wikitable plainrowheaders collapsible' (a partir de la tercera tabla, como ya se mencionó), estableciendo la variable 'table' en la tabla actual en cada iteración.
'''
for table_number, table in enumerate(soup.find_all('table', 'wikitable plainrowheaders collapsible')):
    '''
    El segundo ciclo for recorre todas las filas (etiquetadas como 'tr') dentro de la tabla actual. En cada iteración, la variable 'rows' se establece en la fila actual. 
    '''
    for rows in table.find_all('tr'):
        '''
        Dentro del segundo ciclo hay un un condicional que verifica si la fila contiene una etiqueta th:
            * Si contiene una etiqueta th, se comprueba si esta contiene un string.
            * Si contiene un string, se establece la variable flight_number con el valor del string, y se comprueba si este valor es un número.
            * Si no es un número o no contiene un string, se establece una variable flag en false.
        '''
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        row = rows.find_all('td') # Guardar en la variable row las filas etiquetadas como td.
        if flag: # Si la variable flag es True, se aumenta extracted_row en 1.
            extracted_row += 1
            # Flight Number value.
            # TODO: Añadir el valor de flight_number a la clave 'Flight No.' del diccionario launch_dict.
            print(flight_number)
            launch_dict['Flight No.'] = flight_number
            datatimelist = date_time(row[0]) # Llamar a la función auxiliar date_time() pasando como argumento el primer elemento de la fila que se había guardado previamente en la variable row.

            # Date value
            # TODO: Append the date into launch_dict with key `Date` 
            date = datatimelist[0].strip(',')
            print(date)
            launch_dict['Date'].append(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            print(time)
            launch_dict['Time'].append(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv = booster_version(row[1]) # Función auxiliar.
            if not(bv):
                bv = row[1].a.string
            print(bv)
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            # TODO: Append the launch_site into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            print(launch_site)
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            print(payload)
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4]) # Función auxiliar.
            print(payload_mass)
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            print(orbit)
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            if row[6].a != None:
                customer = row[6].a.string
            else: 
                customer = 'None'
            print(customer)
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            print(launch_outcome)
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            # TODO: Append the booster_landing into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8]) # Función auxiliar.
            print(booster_landing)
            launch_dict['Booster landing'].append(booster_landing)

1
4 June 2010
18:45
F9 v1.0B0003.1
CCAFS
Dragon Spacecraft Qualification Unit
0
LEO
SpaceX
Success

Failure
2
8 December 2010
15:43
F9 v1.0B0004.1
CCAFS
Dragon
0
LEO
NASA
Success
Failure
3
22 May 2012
07:44
F9 v1.0B0005.1
CCAFS
Dragon
525 kg
LEO
NASA
Success
No attempt

4
8 October 2012
00:35
F9 v1.0B0006.1
CCAFS
SpaceX CRS-1
4,700 kg
LEO
NASA
Success

No attempt
5
1 March 2013
15:10
F9 v1.0B0007.1
CCAFS
SpaceX CRS-2
4,877 kg
LEO
NASA
Success

No attempt

6
29 September 2013
16:00
F9 v1.1B1003
VAFB
CASSIOPE
500 kg
Polar orbit
MDA
Success
Uncontrolled
7
3 December 2013
22:41
F9 v1.1
CCAFS
SES-8
3,170 kg
GTO
SES
Success
No attempt
8
6 January 2014
22:06
F9 v1.1
CCAFS
Thaicom 6
3,325 kg
GTO
Thaicom
Success
No attempt
9
18 April 2014
19:25
F9 v1.1
Cape Canaveral
SpaceX CRS-3
2,296 kg
LEO
NASA
Success

Controlled
10
14 July 2014
15:15
F9 v1.1
Cape Canaveral
Orbcomm-OG2
1,316 kg
LEO
Orbcomm
Success
Controlled
11
5 August 2014
08:00
F9 v1.1
Cape Canaveral
AsiaSat 8
4,535 kg
GTO
AsiaSat
Succes

In [17]:
# Finalmente, crear un DataFrame con el launch_dict completo.
df = pd.DataFrame(launch_dict)
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,121,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,121,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,121,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,121,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,121,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10


Por último, exportar el dataset a un CSV para la siguiente sección.

In [19]:
df.to_csv('../datasets/dataset_part_2.csv', index=False)

> **NOTA:**
Para que las respuestas sean consistentes en caso de que que hayan dificultades, los siguientes archivos utilizarán un dataset proporcionado para que cada cuaderno sea independiente.

## Resumen

El desarrollo de este cuaderno puede resumirse en los siguientes puntos:

+ El primer paso fue Importar las librerías necesarias y definir funciones auxiliares para poder procesar la <strong>tabla HTML objetivo</strong>, tratandose esta de datos sobre los lanzamientos de cohetes Falcon 9.
+ Luego se utilizó la url de la Wiki a raspar para hacer una solictud <code>GET</code> y se asignó la respuesta a un objeto <code>response</code>, cuyo texto sirvió para crear un objeto <code>BeautifulSoup</code> (soup).
+ Se encontraron los <strong>registros de lanzamientos de Falcon 9</strong> a partir de la <strong>tercera tabla</strong> HTML, por lo cual se la usó para obtener sus elementos <code>`<th>`</code>, iterar sobre ellos y aplicarles la función auxiliar <code>extract_column_from_header()</code> para extraer uno por uno los nombres de las columnas de dicha tabla.
+ Los nombres extraídos en el punto anterior, se usaron para crear las llaves de un diccionario llamado <code>launch_dict</code>, al que se le atribuyeron listas vacias como valores.
+ Las listas vacias se completaron con un código encargado de parsear los registros de las filas de cada tabla sobre lanzamientos. El código además hizo uso de las funciones auxiliares. Todo esto dió como resultado un diccionario con las columnas de la tabla como claves y las listas de filas como valores. Se creó un DataFrame con esto.
+ Finalmente, importé el dataset final como un archivo CSV para usarlo en la siguiente sección.

---